In [24]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import time
import requests
import pandas
from bs4 import BeautifulSoup
import json
import re
import numpy
import itertools
import pandas as pd
from unidecode import unidecode
from time import sleep
from IPython.display import clear_output

In [25]:
url = 'https://www.hbogola.com/home'

In [33]:
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")#Driver de crome
driver.get(url)# Aca utilizamos un get para el driver
driver.maximize_window()#maximizamos la ventana
cookies_xpath = '//*[@id="onetrust-accept-btn-handler"]'#Identificamos las cookies
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, cookies_xpath))).click()#Aceptamos las cookies
series_xpath = '//*[@id="root"]/div/div[1]/div/div[1]/div/ul/li[3]/a/span'
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, series_xpath))).click()#Aceptamos las cookies
storeby_xpath = '//*[@id="root"]/div/div[2]/div/div/div[2]/div/div/div/span'

In [27]:
def get_serie_info(url):
    """
    Esta función recibe la url de una serie
    y devuelve un diccionario con la metadata asociada.
    """
    # Get de la url
    #driver.get(url)
    
    # Esperamos a que haya cargado la página
    WebDriverWait(driver, 5).until(lambda d: d.find_element_by_tag_name("p"))
    
    ## Buscamos los distintos elementos a partir de su class name
    # titleOfMovie
    title = driver.find_element(By.CLASS_NAME, "titleOfMovie")
    
    # detailsElement
    details = driver.find_elements(By.CLASS_NAME, "detailsElement")
    # Extraemos el contenido de cada elemento de details y normalizamos el texto
    details = [unidecode(detail.text) for detail in details]
    
    # abstract
    abstract = driver.find_element(By.CLASS_NAME, "abstract")
    
    
    # Creamos un diccionario donde ir volcando la info de la película
    serie = dict()
    serie['title'] = title.text
    serie['details'] = details
    serie['abstract'] = abstract.text
    
    return serie

In [28]:
series = list(); error_urls = list()

In [29]:
letters = ['#', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
           'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
ids = list(range(3, len(letters)+3))
letters_ids = dict(zip(letters, ids))
letters_ids

{'#': 3,
 'A': 4,
 'B': 5,
 'C': 6,
 'D': 7,
 'E': 8,
 'F': 9,
 'G': 10,
 'H': 11,
 'I': 12,
 'J': 13,
 'K': 14,
 'L': 15,
 'M': 16,
 'N': 17,
 'O': 18,
 'P': 19,
 'Q': 20,
 'R': 21,
 'S': 22,
 'T': 23,
 'U': 24,
 'V': 25,
 'W': 26,
 'X': 27,
 'Y': 28,
 'Z': 29}

In [30]:
def get_series_by_letter_id(letter_id):
    x = letter_id
    y = 1
    z = 1
    while True:
        try:
            driver.find_element(By.XPATH, f'//*[@id="root"]/div/div[2]/div/div/div[{x}]/div/div/div/div[{y}]/div[{z}]/div/img').click()
            sleep(3)
            current_url = driver.current_url
            try:
                serie = get_serie_info(current_url)
                series.append(serie)
            except:
                print(f'Error en {current_url}')
                error_urls.append(current_url)
            driver.back()
            sleep(3)
            z += 1
        except:
            y += 1
            z = 1
            try:
                driver.find_element(By.XPATH, f'//*[@id="root"]/div/div[2]/div/div/div[{x}]/div/div/div/div[{y}]/div[{z}]/div/img').click()
                sleep(3)
                current_url = driver.current_url
                try:
                    serie = get_serie_info(current_url)
                    series.append(serie)
                except:
                    print(f'Error en {current_url}')
                    error_urls.append(current_url)
                driver.back()
                sleep(3)
                z += 1
            except:
                break

In [34]:
for letter_id in ids:
    try:
        get_series_by_letter_id(letter_id)
    except:
        pass

In [35]:
series

[{'title': '2 Dope Queens - Season 1',
  'details': ['Seasons 1', '2018', 'R'],
  'abstract': 'Jessica Williams and Phoebe Robinson bring their hit comedy podcast back to HBO with four all-new specials, featuring more of the pair’s insights on sex, romance, race, fashion, music, hair journeys, and much more. With no subject off limits, 2 Dope Queens offers up the laugh-out-loud banter of this unique comedy duo, along with their favorite comedians and celebrity guests.'},
 {'title': 'A Black Lady Sketch Show - Season 2',
  'details': ['Seasons 2', '2020', 'R'],
  'abstract': 'Oh, you thought a pandemic was gonna stop us? The Emmy®-nominated sketch show returns to the screens of HBO for a second season featuring a core cast of Black women living relatable, hilarious experiences in a magical reality that subverts traditional expectations.'},
 {'title': '2 Dope Queens - Season 1',
  'details': ['Seasons 1', '2018', 'R'],
  'abstract': 'Jessica Williams and Phoebe Robinson bring their hit c

In [36]:
with open('series.json', 'w') as file:
    json.dump(series, file)